In [1]:
import requests
import json
import re
import time
import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs

In [2]:
# post_urls = ['https://glowpick.com/products/151498','https://glowpick.com/products/154500',
#             'https://www.glowpick.com/products/33543']

def get_post_info(keyword, post_urls):
    driver = webdriver.Chrome()
    color_type_list = []
    tags_list = []
    detail_list = []
    seller_list = []
    category_list = []
    for post_url in post_urls:
        print(post_url)
        driver.maximize_window()
        driver.get(post_url)
        driver.implicitly_wait(20)

        #팝업1 제거
        try:
            driver.find_element_by_css_selector(
                '#default-layout > div > div.modals > span > div.popup > div.popup__container > div.popup__container__buttons.buttons > button.buttons__button.buttons__close').click()
        except:
            None
        # 제품 설명 전체보기 클릭 (제품 상세정보 크롤링)
        driver.find_element_by_css_selector(
            '#contents > section > div.product__info.info > article.info__article.description > h3 > button').send_keys(
            Keys.ENTER)
        # 컬러/타입
        try:
            color_type = driver.find_element_by_css_selector(
                "#default-layout > div > div.modals > span > div > div.details__contents.awards > div > article:nth-child(2) > div").text
        except:
            color_type = None
        color_type_list.append(color_type)
        # 태그
        try:
            tags = ", ".join([x.text for x in driver.find_elements_by_class_name("descriptions__article__keywords")])
        except AttributeError:
            tags = None
        tags_list.append(tags)
        # 소분류 카테고리
        try:
            category = driver.find_element_by_css_selector(
                '#default-layout > div > div.modals > span > div > div.details__contents.awards > div > article:nth-child(3) > p > span:nth-child(3)').text
        except:
            category = None
        category_list.append(category)
        # 제품 상세 설명
        try:
            detail = driver.find_element_by_css_selector(
                '#default-layout > div > div.modals > span > div > div.details__contents.awards > div > article:nth-child(1) > pre').text
        except:
            detail = None
        detail_list.append(detail)
        # 판매처
        try:
            sellers = ",".join([x.text for x in driver.find_elements_by_class_name("stores__store__name")])
        except:
            sellers = None
        seller_list.append(sellers)

        # 제품 설명 전체보기 닫기
        driver.find_element_by_css_selector(
            '#default-layout > div > div.modals > span > div > div.details__contents.awards > h1 > button').send_keys(
            Keys.ENTER)
        
    time.sleep(2)
    driver.quit()
    dict1 = pd.DataFrame({'color_type': color_type_list, 'seller':seller_list, 'tag': tags_list,
                          'detail':detail_list,'category':category_list})
    return dict1